<a href="https://colab.research.google.com/github/david132313/A_shareStock_data_and_model/blob/main/%E6%95%B0%E6%8D%AE%E5%BA%93startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 0：挂载 Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cell 1：路径

In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/AshareDB"
DAILY_DIR = os.path.join(BASE_DIR, "daily_parquet")     # 你的 parquet 目录
DB_DIR    = os.path.join(BASE_DIR, "db")               # 你的 db 目录

os.makedirs(DAILY_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

DB_DRIVE = os.path.join(DB_DIR, "ashare.sqlite")       # 最终输出
DB_LOCAL = "/content/ashare.sqlite"                    # 本地写库（快）

# 清理 Drive 上旧库（避免 WAL 残留）
for fp in [DB_DRIVE, DB_DRIVE+"-wal", DB_DRIVE+"-shm", DB_DRIVE+".tmp"]:
    if os.path.exists(fp):
        os.remove(fp)
        print("deleted:", fp)

# 清理本地旧库
for fp in [DB_LOCAL, DB_LOCAL+"-wal", DB_LOCAL+"-shm"]:
    if os.path.exists(fp):
        os.remove(fp)
        print("deleted:", fp)

print("DAILY_DIR =", DAILY_DIR)
print("DB_DRIVE  =", DB_DRIVE)
print("DB_LOCAL  =", DB_LOCAL)


deleted: /content/ashare.sqlite
deleted: /content/ashare.sqlite-wal
deleted: /content/ashare.sqlite-shm
DAILY_DIR = /content/drive/MyDrive/AshareDB/daily_parquet
DB_DRIVE  = /content/drive/MyDrive/AshareDB/db/ashare.sqlite
DB_LOCAL  = /content/ashare.sqlite


In [ ]:
import os, re

DATE_RE = re.compile(r"^daily_(\d{8})\.parquet$")
files = []
for fn in os.listdir(DAILY_DIR):
    m = DATE_RE.match(fn)
    if m:
        files.append((m.group(1), os.path.join(DAILY_DIR, fn)))
files.sort()

print("num parquet files:", len(files))
if files:
    print("first:", files[0][0], "last:", files[-1][0])
else:
    raise RuntimeError("No parquet files found in DAILY_DIR.")


num parquet files: 6293
first: 20000104 last: 20251219


In [ ]:
import sqlite3

def connect_local_sqlite(path: str) -> sqlite3.Connection:
    conn = sqlite3.connect(path, timeout=60)
    conn.isolation_level = None
    conn.execute("PRAGMA journal_mode=WAL;")       # 本地 WAL 快
    conn.execute("PRAGMA synchronous=NORMAL;")
    conn.execute("PRAGMA temp_store=MEMORY;")
    conn.execute("PRAGMA cache_size=-200000;")
    conn.execute("PRAGMA busy_timeout=60000;")
    conn.execute("PRAGMA locking_mode=EXCLUSIVE;")
    return conn

SCHEMA_NO_INDEX = """
CREATE TABLE IF NOT EXISTS security_map (
  sec_id  INTEGER PRIMARY KEY AUTOINCREMENT,
  ts_code TEXT NOT NULL UNIQUE
);

CREATE TABLE IF NOT EXISTS daily_price (
  trade_date INTEGER NOT NULL,   -- YYYYMMDD
  sec_id     INTEGER NOT NULL,
  open       REAL,
  high       REAL,
  low        REAL,
  close      REAL,
  pre_close  REAL,
  change     REAL,
  pct_chg    REAL,
  vol        REAL,
  amount     REAL,
  PRIMARY KEY (trade_date, sec_id)
);

CREATE TABLE IF NOT EXISTS ingest_manifest (
  trade_date   INTEGER PRIMARY KEY,
  parquet_file TEXT NOT NULL,
  rows         INTEGER NOT NULL,
  status       TEXT NOT NULL,     -- ok/fail
  message      TEXT,
  loaded_at    TEXT NOT NULL
);
"""

conn = connect_local_sqlite(DB_LOCAL)
conn.executescript(SCHEMA_NO_INDEX)

print("local DB created:", DB_LOCAL)
print(conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall())


local DB created: /content/ashare.sqlite
[('daily_price',), ('ingest_manifest',), ('security_map',), ('sqlite_sequence',)]


In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import sqlite3, os

UPSERT_DAILY_SQL = """
INSERT INTO daily_price(
  trade_date, sec_id, open, high, low, close, pre_close, change, pct_chg, vol, amount
) VALUES (?,?,?,?,?,?,?,?,?,?,?)
ON CONFLICT(trade_date, sec_id) DO UPDATE SET
  open=excluded.open,
  high=excluded.high,
  low=excluded.low,
  close=excluded.close,
  pre_close=excluded.pre_close,
  change=excluded.change,
  pct_chg=excluded.pct_chg,
  vol=excluded.vol,
  amount=excluded.amount;
"""

PARQUET_COLS = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount']
SQL_COLS     = ["trade_date","sec_id","open","high","low","close","pre_close","change","pct_chg","vol","amount"]

def chunked(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

# 映射缓存（几千条，放内存很小）
cache_ts2id = {}
for ts_code, sec_id in conn.execute("SELECT ts_code, sec_id FROM security_map;"):
    cache_ts2id[ts_code] = sec_id

def ensure_sec_ids_cached(conn: sqlite3.Connection, codes: list[str]):
    codes = [str(x) for x in codes]
    new_codes = [c for c in codes if c not in cache_ts2id]
    if not new_codes:
        return

    conn.executemany("INSERT OR IGNORE INTO security_map(ts_code) VALUES (?);",
                     [(c,) for c in set(new_codes)])

    uniq = sorted(set(new_codes))
    for batch in chunked(uniq, 900):
        placeholders = ",".join(["?"] * len(batch))
        q = f"SELECT ts_code, sec_id FROM security_map WHERE ts_code IN ({placeholders});"
        for ts_code, sec_id in conn.execute(q, batch):
            cache_ts2id[ts_code] = sec_id

def backup_local_to_drive(local_conn: sqlite3.Connection, drive_path: str):
    # 用 sqlite backup 生成“干净的主库文件”，避免 -wal/-shm 残留在 Drive
    tmp = drive_path + ".tmp"
    dst = sqlite3.connect(tmp)
    try:
        local_conn.backup(dst)
        dst.commit()
    finally:
        dst.close()
    os.replace(tmp, drive_path)

# 断点：跳过已 ok 的日期
loaded_ok = set(r[0] for r in conn.execute(
    "SELECT trade_date FROM ingest_manifest WHERE status='ok';"
).fetchall())
todo = [(d,p) for (d,p) in files if int(d) not in loaded_ok]
print("already ok:", len(loaded_ok), "| remaining:", len(todo), "| next:", (todo[0][0] if todo else None))

SAVE_EVERY_N = 22  # 每 ~1 个月备份一次到 Drive（你可以改大一点更省 Drive I/O）
ok_n = fail_n = 0

for i, (d, p) in enumerate(tqdm(todo, desc="ingest"), start=1):
    parquet_name = os.path.basename(p)
    trade_date_int = int(d)
    try:
        df = pd.read_parquet(p)

        missing = [c for c in PARQUET_COLS if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns {missing} in {parquet_name}")

        df["trade_date"] = df["trade_date"].astype(str).astype(int)

        conn.execute("BEGIN;")

        codes = df["ts_code"].astype(str).unique().tolist()
        ensure_sec_ids_cached(conn, codes)
        df["sec_id"] = df["ts_code"].astype(str).map(cache_ts2id).astype(int)

        df2 = df[SQL_COLS].where(pd.notnull(df[SQL_COLS]), None)

        # sanity（防止写反）
        td0 = int(df2.iloc[0,0]); sid0 = int(df2.iloc[0,1])
        if td0 < 19900101 or td0 > 21000101:
            raise ValueError(f"Sanity failed trade_date: {td0}")
        if sid0 <= 0:
            raise ValueError(f"Sanity failed sec_id: {sid0}")

        conn.executemany(UPSERT_DAILY_SQL, df2.itertuples(index=False, name=None))

        conn.execute("""
        INSERT OR REPLACE INTO ingest_manifest(trade_date, parquet_file, rows, status, message, loaded_at)
        VALUES (?,?,?,?,?,?)
        """, (trade_date_int, parquet_name, int(len(df2)), "ok", None,
              datetime.now().isoformat(timespec="seconds")))

        conn.execute("COMMIT;")
        ok_n += 1

    except Exception as e:
        try:
            conn.execute("ROLLBACK;")
        except Exception:
            pass
        conn.execute("BEGIN;")
        conn.execute("""
        INSERT OR REPLACE INTO ingest_manifest(trade_date, parquet_file, rows, status, message, loaded_at)
        VALUES (?,?,?,?,?,?)
        """, (trade_date_int, parquet_name, 0, "fail", repr(e),
              datetime.now().isoformat(timespec="seconds")))
        conn.execute("COMMIT;")
        fail_n += 1

    if i % SAVE_EVERY_N == 0:
        backup_local_to_drive(conn, DB_DRIVE)
        print(f"\n[checkpoint] saved to Drive at {d} | ok={ok_n} fail={fail_n} | cache={len(cache_ts2id)}")

# 最后备份一次
backup_local_to_drive(conn, DB_DRIVE)
print("DONE. ok:", ok_n, "fail:", fail_n, "saved to:", DB_DRIVE)
print("integrity_check:", conn.execute("PRAGMA integrity_check;").fetchone()[0])


already ok: 0 | remaining: 6293 | next: 20000104


ingest:   0%|          | 26/6293 [00:03<09:06, 11.46it/s]


[checkpoint] saved to Drive at 20000216 | ok=22 fail=0 | cache=931


ingest:   1%|          | 49/6293 [00:03<03:58, 26.15it/s]


[checkpoint] saved to Drive at 20000317 | ok=44 fail=0 | cache=938


...


[checkpoint] saved to Drive at 20251118 | ok=6270 fail=0 | cache=5768


ingest: 100%|█████████▉| 6292/6293 [2:08:15<00:18, 18.83s/it]


[checkpoint] saved to Drive at 20251218 | ok=6292 fail=0 | cache=5778


ingest: 100%|██████████| 6293/6293 [2:08:16<00:00,  1.22s/it]


DONE. ok: 6293 fail: 0 saved to: /content/drive/MyDrive/AshareDB/db/ashare.sqlite
integrity_check: ok


In [ ]:
conn.execute("BEGIN;")
conn.execute("CREATE INDEX IF NOT EXISTS idx_daily_date     ON daily_price(trade_date);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_daily_sec_date ON daily_price(sec_id, trade_date);")
conn.execute("COMMIT;")

# 再备份一次到 Drive（带上索引）
import os, sqlite3
tmp = DB_DRIVE + ".tmp"
dst = sqlite3.connect(tmp)
conn.backup(dst)
dst.commit()
dst.close()
os.replace(tmp, DB_DRIVE)

print("indexes built and saved to Drive:", DB_DRIVE)
print(conn.execute("PRAGMA integrity_check;").fetchone()[0])


indexes built and saved to Drive: /content/drive/MyDrive/AshareDB/db/ashare.sqlite
ok


In [ ]:
import pandas as pd

print(pd.read_sql_query("""
SELECT MIN(trade_date) AS min_date, MAX(trade_date) AS max_date, COUNT(*) AS total_rows
FROM daily_price;
""", conn))

print(pd.read_sql_query("""
SELECT status, COUNT(*) AS n
FROM ingest_manifest
GROUP BY status;
""", conn))

print(pd.read_sql_query("""
SELECT trade_date, parquet_file, message
FROM ingest_manifest
WHERE status='fail'
ORDER BY trade_date
LIMIT 20;
""", conn))

conn.close()


   min_date  max_date  total_rows
0  20000104  20251219    16430270
  status     n
0     ok  6293
Empty DataFrame
Columns: [trade_date, parquet_file, message]
Index: []


# 测试 1：确认每天大概有多少股票（抽样 10 天）

In [ ]:
import sqlite3, pandas as pd, numpy as np

DB = "/content/drive/MyDrive/AshareDB/db/ashare.sqlite"
conn = sqlite3.connect(f"file:{DB}?mode=ro", uri=True)

# 抽 10 个交易日
dates = pd.read_sql_query("SELECT DISTINCT trade_date FROM daily_price ORDER BY trade_date;", conn)["trade_date"].values
sample = np.linspace(0, len(dates)-1, 10, dtype=int)
sample_dates = dates[sample].tolist()

df = pd.read_sql_query(f"""
SELECT trade_date, COUNT(*) AS n_stocks
FROM daily_price
WHERE trade_date IN ({",".join(["?"]*len(sample_dates))})
GROUP BY trade_date
ORDER BY trade_date;
""", conn, params=sample_dates)

conn.close()
df


,trade_date,n_stocks
0,20000104,913
1,20021209,1176
2,20051101,1233
3,20080911,1524
4,20110729,2125
5,20140623,2308
6,20170504,3017
7,20200317,3851
8,20230203,5068
9,20251219,5448


测试 2：随机抽一只股票，拉它完整历史（检查索引是否好用）

In [ ]:
import sqlite3, pandas as pd, random, time

DB = "/content/drive/MyDrive/AshareDB/db/ashare.sqlite"
conn = sqlite3.connect(f"file:{DB}?mode=ro", uri=True)

# 随机抽一个 sec_id
sec_id = conn.execute("SELECT sec_id FROM security_map ORDER BY RANDOM() LIMIT 1;").fetchone()[0]
ts_code = conn.execute("SELECT ts_code FROM security_map WHERE sec_id=?;", (sec_id,)).fetchone()[0]

t0 = time.perf_counter()
hist = pd.read_sql_query("""
SELECT trade_date, open, high, low, close, vol, amount
FROM daily_price
WHERE sec_id=?
ORDER BY trade_date;
""", conn, params=(sec_id,))
t1 = time.perf_counter()

conn.close()
print("ts_code:", ts_code, "rows:", len(hist), "query_sec_history_time_sec:", round(t1-t0, 3))
hist.head()


ts_code: 601857.SH rows: 4399 query_sec_history_time_sec: 6.634


,trade_date,open,high,low,close,vol,amount
0,20071105,48.60,48.62,41.70,43.96,15474994.87,6.999139e+07
1,20071106,41.40,42.40,39.80,39.99,3429778.20,1.400025e+07
2,20071107,39.70,40.73,38.28,40.43,2166188.70,8.575267e+06
3,20071108,39.20,39.75,38.00,38.19,1400509.61,5.447045e+06
4,20071109,37.85,38.39,36.66,38.18,1432266.03,5.379485e+06


测试 3：查某一天全市场数据并做一个简单 alpha（不需要历史）

In [ ]:
import sqlite3, pandas as pd, time

DB = "/content/drive/MyDrive/AshareDB/db/ashare.sqlite"
conn = sqlite3.connect(f"file:{DB}?mode=ro", uri=True)

# 取最后一个交易日
d = pd.read_sql_query("SELECT MAX(trade_date) AS d FROM daily_price;", conn)["d"].iloc[0]

t0 = time.perf_counter()
day = pd.read_sql_query("""
SELECT dp.sec_id, sm.ts_code, dp.close, dp.pct_chg, dp.vol, dp.amount
FROM daily_price dp
JOIN security_map sm ON sm.sec_id=dp.sec_id
WHERE dp.trade_date=?;
""", conn, params=(int(d),))
t1 = time.perf_counter()

conn.close()
print("trade_date:", d, "rows:", len(day), "query_time_sec:", round(t1-t0, 3))

# 一个简单 alpha：流动性 alpha = log(amount)
import numpy as np
day["alpha"] = np.log(day["amount"].clip(lower=1))
day = day.replace([np.inf, -np.inf], np.nan).dropna(subset=["alpha"])

day.sort_values("alpha", ascending=False).head(10)[["ts_code","alpha","amount","pct_chg"]]


trade_date: 20251219 rows: 5448 query_time_sec: 0.074


,ts_code,alpha,amount,pct_chg
1946,300502.SZ,16.564123,1.562094e+07,2.0682
1767,300308.SZ,16.378848,1.297906e+07,0.0875
125,000547.SZ,16.179395,1.063217e+07,3.9627
3824,601933.SH,16.111256,9.931838e+06,4.9180
3545,600879.SH,16.069845,9.528948e+06,4.5718
2956,600118.SH,15.913131,8.146760e+06,4.2799
1841,300394.SZ,15.884153,7.914076e+06,-2.1845
1737,300274.SZ,15.794011,7.231889e+06,1.6986
4440,603986.SH,15.766391,7.034877e+06,-1.9772
2181,300750.SZ,15.698452,6.572808e+06,0.5376


测试 4：带历史窗口的 alpha（处理“历史不够”的股票）

In [ ]:
import sqlite3, pandas as pd, numpy as np

DB = "/content/drive/MyDrive/AshareDB/db/ashare.sqlite"
conn = sqlite3.connect(f"file:{DB}?mode=ro", uri=True)

# 选一个测试日期：用最新日期
d = pd.read_sql_query("SELECT MAX(trade_date) AS d FROM daily_price;", conn)["d"].iloc[0]
d = int(d)

# 拉过去 60 个交易日的截面（够你算 20 日动量）
# 先取最近 60 个交易日列表
dates = pd.read_sql_query("""
SELECT DISTINCT trade_date
FROM daily_price
WHERE trade_date <= ?
ORDER BY trade_date DESC
LIMIT 60;
""", conn, params=(d,))["trade_date"].tolist()
dates = sorted(dates)

# 拉这 60 天的 close
q = f"""
SELECT trade_date, sec_id, close
FROM daily_price
WHERE trade_date IN ({",".join(["?"]*len(dates))});
"""
px = pd.read_sql_query(q, conn, params=dates)

conn.close()

# pivot: index=trade_date, columns=sec_id
pivot = px.pivot(index="trade_date", columns="sec_id", values="close").sort_index()

# 20日动量
mom20 = pivot.iloc[-1] / pivot.iloc[-21] - 1   # 需要至少 21 行
mom20 = mom20.dropna()

print("alpha computed date:", d, "| valid stocks:", len(mom20), "/", pivot.shape[1])

mom20.sort_values(ascending=False).head(10)


alpha computed date: 20251219 | valid stocks: 5431 / 5464


,0
sec_id,
4644,2.167894
2195,1.399421
5164,1.340739
2700,1.334831
1520,1.334158
844,1.215962
5450,1.146828
1228,1.145556
271,1.028153
